# How to Handle Missing Data
If you want to type along with me, use [this notebook](https://humboldt.cloudbank.2i2c.cloud/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fbethanyj0%2Fdata271_sp25&branch=main&urlpath=tree%2Fdata271_sp25%2Flectures%2Fdata271_lec36_live.ipynb) instead. 
If you don't want to type and want to follow along just by executing the cells, stay in this notebook. 

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(3)

In [ ]:
# Read in example dataset (fabricated for this demo)
df = pd.read_csv('gender_wage_full_data.csv')
df.head()

In [ ]:
# Explore the data 
sns.boxplot(data = df, x = 'gender', y = 'salary');

In [ ]:
# Function for permutation test to determine if difference in means is significant
def permutation_test(data, group_col, value_col):
    observed_diff = data.groupby(group_col)[value_col].mean().diff().iloc[-1]
    diffs = []
    for i in range(5000):
        shuffled = data[value_col].sample(frac=1, replace=False).reset_index(drop=True)
        shuffled_df = data.copy()
        shuffled_df[value_col] = shuffled
        diff = shuffled_df.groupby(group_col)[value_col].mean().diff().iloc[-1]
        diffs.append(diff)
    p_val = np.mean(np.abs(diffs) >= np.abs(observed_diff))
    return observed_diff, p_val, diffs

In [ ]:
# Do the permutation test
obs_diff, p_val, diffs = permutation_test(df, "gender", "salary")

In [ ]:
# Visualize results
sns.histplot(diffs)
plt.scatter(x=obs_diff, y = 5, marker='o',color='r');
p_val

What if the dataset had missing values?

In [ ]:
# Randomly remove 15% of salaries
n = df.shape[0]
data_random_missing = df.copy()
mask = np.random.rand(n) < 0.15
data_random_missing.loc[mask, 'salary'] = np.nan
data_random_missing.head(10)

In [ ]:
# Drop the rows with missing salary 
data_random_missing_orig = data_random_missing.copy()
data_random_missing.dropna(subset='salary',inplace=True)
data_random_missing.shape

In [ ]:
# Do the permutation test
obs_diff_random, p_val_random, diffs_random = permutation_test(data_random_missing, "gender", "salary")

In [ ]:
# Visualize results
sns.histplot(diffs_random)
plt.scatter(x=obs_diff_random, y = 5, marker='o',color='r');
p_val_random

What if the missingness is not random? Suppose it is based on gender/age.

In [ ]:
# Generate missing data based on gender/age 
# (young females less likely to have data)
missing_based_on_gender_age = df.copy()
mask = ((missing_based_on_gender_age['gender'] == 'female') 
        & (missing_based_on_gender_age['age'] < 30) 
        & (np.random.rand(n) < 0.5))
missing_based_on_gender_age.loc[mask, 'salary'] = np.nan

In [ ]:
# Drop the rows with missing salary 
missing_based_on_gender_age_orig = missing_based_on_gender_age.copy()
missing_based_on_gender_age.dropna(subset='salary',inplace=True)
missing_based_on_gender_age.shape

In [ ]:
# Do the permutation test
obs_diff_missing, p_val_missing, diffs_missing = permutation_test(missing_based_on_gender_age, "gender", "salary")

In [ ]:
# Visualize results
sns.histplot(diffs_missing)
plt.scatter(x=obs_diff_missing, y = 5, marker='o',color='r');
p_val_missing

What if missingness is based on the value itself?

In [ ]:
# Generate missing data based on salary itself. Low salaries are more likely to skip
missing_based_on_salary = df.copy()
mask = (missing_based_on_salary['salary'] < 55000) & (np.random.rand(n) < 0.7)
missing_based_on_salary.loc[mask, 'salary'] = np.nan

In [ ]:
# Drop the rows with missing salary 
missing_based_on_salary_orig = missing_based_on_salary.copy()
missing_based_on_salary.dropna(subset='salary',inplace=True)
missing_based_on_salary.shape

In [ ]:
# Do the permutation test
obs_diff_missing2, p_val_missing2, diffs_missing2 = permutation_test(missing_based_on_salary, "gender", "salary")

In [ ]:
# Visualize results
sns.histplot(diffs_missing2)
plt.scatter(x=obs_diff_missing2, y = 5, marker='o',color='r');
p_val_missing2

### Checking for patterns in missingness

In [ ]:
# Create a "missing salary" column
data_random_missing_orig['missing_salary'] = data_random_missing_orig.salary.isna()
missing_based_on_gender_age_orig['missing_salary'] = missing_based_on_gender_age_orig.salary.isna()
missing_based_on_salary_orig['missing_salary'] = missing_based_on_salary_orig.salary.isna()

In [ ]:
# check relationship between missingness and gender, MCAR
data_random_missing_orig.groupby('gender')['missing_salary'].mean()

In [ ]:
# check relationship between missingness and age, MCAR
sns.violinplot(x='missing_salary', y='age', data=data_random_missing_orig);

In [ ]:
# check relationship between missingness and gender, MAR
missing_based_on_gender_age_orig.groupby('gender')['missing_salary'].mean()

In [ ]:
# check relationship between missingness and age, MAR
sns.violinplot(x='missing_salary', y='age', data=missing_based_on_gender_age_orig);

In [ ]:
# check relationship between missingness and gender, MNAR
missing_based_on_salary_orig.groupby('gender')['missing_salary'].mean()

In [ ]:
# check relationship between missingness and age, MNAR
sns.violinplot(x='missing_salary', y='age', data=missing_based_on_salary_orig);

### Imputing the data

In [ ]:
# Fill with the mean
random_filled = data_random_missing_orig.fillna(data_random_missing_orig.salary.mean())

In [ ]:
# Check analysis
obs_diff, p_val, diffs = permutation_test(random_filled, "gender", "salary")

sns.histplot(diffs)
plt.scatter(x=obs_diff, y = 5, marker='o',color='r');
p_val

In [ ]:
# Fill with the median
random_filled = data_random_missing_orig.fillna(data_random_missing_orig.salary.median())

In [ ]:
# Check analysis
obs_diff, p_val, diffs = permutation_test(random_filled, "gender", "salary")
p_val

In [ ]:
# Fill MAR with the mean
MAR_filled = missing_based_on_gender_age_orig.fillna(
    missing_based_on_gender_age_orig.salary.mean())

In [ ]:
# Check analysis
obs_diff, p_val, diffs = permutation_test(MAR_filled, "gender", "salary")
p_val

In [ ]:
# Fill MAR with the median
MAR_filled = missing_based_on_gender_age_orig.fillna(
    missing_based_on_gender_age_orig.salary.median())

In [ ]:
# Check analysis
obs_diff, p_val, diffs = permutation_test(MAR_filled, "gender", "salary")
p_val

In [ ]:
# Fill MNAR with the mean
MNAR_filled = missing_based_on_salary_orig.fillna(
    missing_based_on_salary_orig.salary.mean())

In [ ]:
# Check analysis
obs_diff, p_val, diffs = permutation_test(MNAR_filled, "gender", "salary")
p_val

In [ ]:
# Fill MNAR with the median
MNAR_filled = missing_based_on_salary_orig.fillna(
    missing_based_on_salary_orig.salary.median())

In [ ]:
# Check analysis
obs_diff, p_val, diffs = permutation_test(MNAR_filled, "gender", "salary")
p_val